# Task 2a
Die folgenden Kapitel widmen sich der Datenvorbereitung der gecrawlten Textabschnitte mit Identifizierung von Named Entities und nicht-NE Touristenattraktionen sowie einem anschließenden Clustering der Dokumente nach den nicht-NE Touristenattraktionen.

----------------------------------

#### Inhaltsverzeichnis:
#### 1. [Text Preprocessing](#Text_prep)
#### 2. [Erstellen von Trainingsdaten](#Build_train_data)
#### 3. [Clustering](#Clustering)
#### 4. [Bonus: Automatisches mapping der NoNE zu Kategorien](#Bonus)
-----------------------------------------

##### Imports

In [1]:
from task2a_preprocessing import Preprocessing
from task2a_clustering import Clustering
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already

In [2]:
import task2a_prep_functions as p

In [3]:
df_raw = pd.read_csv('results_scrapping.csv', index_col=0)
df_raw

,Link,Place,Content
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ..."
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi..."
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...
4,https://www.roughguides.com/usa/great-plains/,The Great Plains Travel Guide,The rolling hills and vast grasslands of the G...
...,...,...,...
95,https://www.roughguides.com/spain/cantabria-an...,The Picos de Europa,The Picos de Europa may not be the highest mou...
96,https://www.roughguides.com/spain/cantabria-an...,Santander,"Easily the largest city in Cantabria, with a p..."
97,https://www.roughguides.com/spain/andalucia/ma...,Málaga,MÁLAGA seems at first an uninviting place. It’...
98,https://www.roughguides.com/spain/the-balearic...,Ibiza,"IBIZA, or Eivissa in Catalan, is an island of ..."


In [4]:
df_mapping = df_raw.copy()
df_mapping['Country'] = [i[3] for i in df_mapping['Link'].str.split('/')]
df_mapping['Continent'] = df_mapping.apply(lambda row: p.label_continent(row), axis=1)
df_mapping

,Link,Place,Content,Country,Continent
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...",usa,America
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...",usa,America
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,usa,America
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,usa,America
4,https://www.roughguides.com/usa/great-plains/,The Great Plains Travel Guide,The rolling hills and vast grasslands of the G...,usa,America
...,...,...,...,...,...
95,https://www.roughguides.com/spain/cantabria-an...,The Picos de Europa,The Picos de Europa may not be the highest mou...,spain,Europe
96,https://www.roughguides.com/spain/cantabria-an...,Santander,"Easily the largest city in Cantabria, with a p...",spain,Europe
97,https://www.roughguides.com/spain/andalucia/ma...,Málaga,MÁLAGA seems at first an uninviting place. It’...,spain,Europe
98,https://www.roughguides.com/spain/the-balearic...,Ibiza,"IBIZA, or Eivissa in Catalan, is an island of ...",spain,Europe


In [5]:
df = df_raw.copy()
df['sentences'] = p.s_tokenize(df['Content'])
df['tokens_full'] = p.w_full_tokenize(df['Content'])
df['tokens_full_lower'] = p.lower_tokens_to_list([df['tokens_full'].tolist()])[0]
df['tokens_full_lower_digi'] = p.keep_only_words([df['tokens_full_lower'].tolist()])[0]
df['lemmas'] = p.lemmatize_to_list([df['tokens_full_lower_digi']])[0]
df['preprocessed'] = p.remove_stopwords([df['lemmas']])[0]
df

,Link,Place,Content,sentences,tokens_full,tokens_full_lower,tokens_full_lower_digi,lemmas,preprocessed
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...","[Built on a reclaimed swamp, two miles east of...","[Built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, two, miles, e...","[build, on, a, reclaim, swamp, two, mile, east...","[build, reclaim, swamp, two, mile, east, downt..."
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...","[Folklore, films and widespread hearsay have g...","[Folklore, ,, films, and, widespread, hearsay,...","[folklore, ,, films, and, widespread, hearsay,...","[folklore, films, and, widespread, hearsay, ha...","[folklore, film, and, widespread, hearsay, hav...","[folklore, film, widespread, hearsay, give, fl..."
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,[America’s oldest and easily its most famous n...,"[America, ’, s, oldest, and, easily, its, most...","[america, ’, s, oldest, and, easily, its, most...","[america, s, oldest, and, easily, its, most, f...","[america, s, old, and, easily, its, most, famo...","[america, old, easily, famous, national, park,..."
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,[Although the Big Island of Hawaii could hold ...,"[Although, the, Big, Island, of, Hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, big, island, hawaii, could, hold, i..."
4,https://www.roughguides.com/usa/great-plains/,The Great Plains Travel Guide,The rolling hills and vast grasslands of the G...,[The rolling hills and vast grasslands of the ...,"[The, rolling, hills, and, vast, grasslands, o...","[the, rolling, hills, and, vast, grasslands, o...","[the, rolling, hills, and, vast, grasslands, o...","[the, rolling, hill, and, vast, grassland, of,...","[rolling, hill, vast, grassland, great, plain,..."
...,...,...,...,...,...,...,...,...,...
95,https://www.roughguides.com/spain/cantabria-an...,The Picos de Europa,The Picos de Europa may not be the highest mou...,[The Picos de Europa may not be the highest mo...,"[The, Picos, de, Europa, may, not, be, the, hi...","[the, picos, de, europa, may, not, be, the, hi...","[the, picos, de, europa, may, not, be, the, hi...","[the, picos, de, europa, may, not, be, the, hi...","[picos, de, europa, may, high, mountain, spain..."
96,https://www.roughguides.com/spain/cantabria-an...,Santander,"Easily the largest city in Cantabria, with a p...","[Easily the largest city in Cantabria, with a ...","[Easily, the, largest, city, in, Cantabria, ,,...","[easily, the, largest, city, in, cantabria, ,,...","[easily, the, largest, city, in, cantabria, wi...","[easily, the, large, city, in, cantabria, with...","[easily, large, city, cantabria, population, a..."
97,https://www.roughguides.com/spain/andalucia/ma...,Málaga,MÁLAGA seems at first an uninviting place. It’...,"[MÁLAGA seems at first an uninviting place., I...","[MÁLAGA, seems, at, first, an, uninviting, pla...","[málaga, seems, at, first, an, uninviting, pla...","[málaga, seems, at, first, an, uninviting, pla...","[málaga, seem, at, first, an, uninviting, plac...","[málaga, seem, first, uninviting, place, secon..."
98,https://www.roughguides.com/spain/the-balearic...,Ibiza,"IBIZA, or Eivissa in Catalan, is an island of ...","[IBIZA, or Eivissa in Catalan, is an island of...","[IBIZA, ,, or, Eivissa, in, Catalan, ,, is, an...","[ibiza, ,, or, eivissa, in, catalan, ,, is, an...","[ibiza, or, eivissa, in, catalan, is, an, isla...","[ibiza, or, eivissa, in, catalan, be, an, isla...","[ibiza, eivissa, catalan, island, excess, bea

In [6]:
df['None'] = [[[]] for i in range(100)]
small, small_plain, full = p.named_entity_recognition_spacy(df['sentences'])
df['named_entities_spacy_small'] = small
df['named_entities_spacy_small_plain'] = small_plain
df['named_entities_spacy_full'] = full
df['NE_preprocessed'] = p.remove_stopwords(p.lemmatize_to_list(p.keep_nouns(p.keep_only_words(p.w_column_tokenize(p.ne_ident_tokens(df[['None','named_entities_spacy_full']]))))))
df['sent_prepcorcessed_only_nouns'] = p.keep_nouns([df['preprocessed']])[0]
df['sent_without_NE'] = p.remove_words_from_sentences(df[['sent_prepcorcessed_only_nouns', 'NE_preprocessed']])
kombs, plain = p.get_hypernyms(df['sent_without_NE'])
df_mapping['no_NE_attractions'] = kombs
df_mapping['no_NE_attractions_plain'] = plain
df['no_NE_attractions_plain'] = plain
unique, unique_dict = p.unique_and_dict_count(df['no_NE_attractions_plain'])
df['no_NE_attractions_plain_unique'] = unique
df['no_NE_attractions_plain_unique_dict'] = unique_dict
df

C:\Users\jenni\.conda\envs\nlp_travel\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Synset('i.e..r.01')
Synset('o.k..n.01')
Synset('c.o.d..r.01')
Synset('c.o.d..r.01')


,Link,Place,Content,sentences,tokens_full,tokens_full_lower,tokens_full_lower_digi,lemmas,preprocessed,None,named_entities_spacy_small,named_entities_spacy_small_plain,named_entities_spacy_full,NE_preprocessed,sent_prepcorcessed_only_nouns,sent_without_NE,no_NE_attractions_plain,no_NE_attractions_plain_unique,no_NE_attractions_plain_unique_dict
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...","[Built on a reclaimed swamp, two miles east of...","[Built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, two, miles, e...","[build, on, a, reclaim, swamp, two, mile, east...","[build, reclaim, swamp, two, mile, east, downt...",[[]],"[[(honolulu, GPE), (waikiki, GPE), (the ala wa...","[[Honolulu, Waikiki, the Ala Wai canal], [], [...","[[two miles, honolulu, waikiki, the ala wai ca...","[[lazy, day], [kalakaua, avenue], [waikiki], [...","[build, reclaim, swamp, mile, east, downtown, ...","[build, reclaim, swamp, east, downtown, island...","[swamp, island, city, sea, point, shop, narrow...","[island, city, point, garden, shop, narrow, se...","{'island': 3, 'city': 1, 'point': 1, 'garden':..."
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...","[Folklore, films and widespread hearsay have g...","[Folklore, ,, films, and, widespread, hearsay,...","[folklore, ,, films, and, widespread, hearsay,...","[folklore, films, and, widespread, hearsay, ha...","[folklore, film, and, widespread, hearsay, hav...","[folklore, film, widespread, hearsay, give, fl...",[[]],"[[(cuba, GPE)], [(key west, GPE), (florida, GP...","[[Cuba], [Key West, Florida, USA, Keys], [the ...","[[hundred-mile, ninety miles, cuba, the 1980s]...","[[cuban], [ninety, mile], [today], [mile], [du...","[folklore, film, hearsay, florida, key, hundre...","[folklore, film, hearsay, chain, island, image...","[chain, island, beach, fishing, dive, coast, s...","[beach, bar, corner, boutique, dive, coast, se...","{'beach': 1, 'bar': 1, 'corner': 1, 'boutique'..."
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,[America’s oldest and easily its most famous n...,"[America, ’, s, oldest, and, easily, its, most...","[america, ’, s, oldest, and, easily, its, most...","[america, s, oldest, and, easily, its, most, f...","[america, s, old, and, easily, its, most, famo...","[america, old, easily, famous, national, park,...",[[]],"[[(america, GPE), (yellowstone national park, ...","[[America, YELLOWSTONE NATIONAL PARK], [Yellow...","[[america, yellowstone national park, three mi...","[[dozen], [canyon], [yellowstone, lake], [thou...","[america, park, yellowstone, park, attract, vi...","[attract, visitor, reason, sheer, diversity, o...","[mountain, meadow, grizzly, bear, bison, buffa...","[course, coyote, dog, bison, wolf, elk, geyser...","{'course': 1, 'coyote': 1, 'dog': 1, 'bison': ..."
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,[Although the Big Island of Hawaii could hold ...,"[Although, the, Big, Island, of, Hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, big, island, hawaii, could, hold, i...",[[]],"[[(the big island of hawaii, LOC)], [(oahu, GP...","[[the Big Island of Hawaii], [Oahu, Maui], [Ko...","[[the big island of hawaii, 185,000, half, cap...","[[eighty], [crater, rim, drive], [island], [oa...","[island, hawaii, hold, island, room, spare, po...","[hold, room, spare, population, town, people, ...","[hold, town, hold, fair, restaurant, bar, town...","[rock, beach, desert, bar, restaurant, resort,...","{'rock': 1, 'beach': 1, 'desert': 1, 'bar': 1,..."
4,https://www.roughguides.com/usa/great-p

In [7]:
df['grams'] = df['no_NE_attractions_plain_unique']
df

,Link,Place,Content,sentences,tokens_full,tokens_full_lower,tokens_full_lower_digi,lemmas,preprocessed,None,named_entities_spacy_small,named_entities_spacy_small_plain,named_entities_spacy_full,NE_preprocessed,sent_prepcorcessed_only_nouns,sent_without_NE,no_NE_attractions_plain,no_NE_attractions_plain_unique,no_NE_attractions_plain_unique_dict,grams
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...","[Built on a reclaimed swamp, two miles east of...","[Built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, two, miles, e...","[build, on, a, reclaim, swamp, two, mile, east...","[build, reclaim, swamp, two, mile, east, downt...",[[]],"[[(honolulu, GPE), (waikiki, GPE), (the ala wa...","[[Honolulu, Waikiki, the Ala Wai canal], [], [...","[[two miles, honolulu, waikiki, the ala wai ca...","[[lazy, day], [kalakaua, avenue], [waikiki], [...","[build, reclaim, swamp, mile, east, downtown, ...","[build, reclaim, swamp, east, downtown, island...","[swamp, island, city, sea, point, shop, narrow...","[island, city, point, garden, shop, narrow, se...","{'island': 3, 'city': 1, 'point': 1, 'garden':...","[island, city, point, garden, shop, narrow, se..."
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...","[Folklore, films and widespread hearsay have g...","[Folklore, ,, films, and, widespread, hearsay,...","[folklore, ,, films, and, widespread, hearsay,...","[folklore, films, and, widespread, hearsay, ha...","[folklore, film, and, widespread, hearsay, hav...","[folklore, film, widespread, hearsay, give, fl...",[[]],"[[(cuba, GPE)], [(key west, GPE), (florida, GP...","[[Cuba], [Key West, Florida, USA, Keys], [the ...","[[hundred-mile, ninety miles, cuba, the 1980s]...","[[cuban], [ninety, mile], [today], [mile], [du...","[folklore, film, hearsay, florida, key, hundre...","[folklore, film, hearsay, chain, island, image...","[chain, island, beach, fishing, dive, coast, s...","[beach, bar, corner, boutique, dive, coast, se...","{'beach': 1, 'bar': 1, 'corner': 1, 'boutique'...","[beach, bar, corner, boutique, dive, coast, se..."
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,[America’s oldest and easily its most famous n...,"[America, ’, s, oldest, and, easily, its, most...","[america, ’, s, oldest, and, easily, its, most...","[america, s, oldest, and, easily, its, most, f...","[america, s, old, and, easily, its, most, famo...","[america, old, easily, famous, national, park,...",[[]],"[[(america, GPE), (yellowstone national park, ...","[[America, YELLOWSTONE NATIONAL PARK], [Yellow...","[[america, yellowstone national park, three mi...","[[dozen], [canyon], [yellowstone, lake], [thou...","[america, park, yellowstone, park, attract, vi...","[attract, visitor, reason, sheer, diversity, o...","[mountain, meadow, grizzly, bear, bison, buffa...","[course, coyote, dog, bison, wolf, elk, geyser...","{'course': 1, 'coyote': 1, 'dog': 1, 'bison': ...","[course, coyote, dog, bison, wolf, elk, geyser..."
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,[Although the Big Island of Hawaii could hold ...,"[Although, the, Big, Island, of, Hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, big, island, hawaii, could, hold, i...",[[]],"[[(the big island of hawaii, LOC)], [(oahu, GP...","[[the Big Island of Hawaii], [Oahu, Maui], [Ko...","[[the big island of hawaii, 185,000, half, cap...","[[eighty], [crater, rim, drive], [island], [oa...","[island, hawaii, hold, island, room, spare, po...","[hold, room, spare, population, town, people, ...","[hold, town, hold, fair, restauran

In [8]:
from nltk import ngrams
def create_n_grams_to_list(column, length):
    """ Create n-grams from tokens

    Args:
        column: Column to transform
        length: Length of the n-grams builded

    Returns:
        n_grams: Column with n-grams
    """

    n_grams_list = []

    for line in column:
        counter = 0
        n_grams = []

        for token_list in line:

            #print(token_list)
            grams = list(ngrams(token_list, length))
            if grams:
                n_grams.append(grams)
            else:
                print()
                print(counter)
            counter+=1

        n_grams_list.append(n_grams)

    return n_grams_list

In [9]:
df['grams'] = create_n_grams_to_list([df['no_NE_attractions_plain_unique']], 2)[0]
df

,Link,Place,Content,sentences,tokens_full,tokens_full_lower,tokens_full_lower_digi,lemmas,preprocessed,None,named_entities_spacy_small,named_entities_spacy_small_plain,named_entities_spacy_full,NE_preprocessed,sent_prepcorcessed_only_nouns,sent_without_NE,no_NE_attractions_plain,no_NE_attractions_plain_unique,no_NE_attractions_plain_unique_dict,grams
0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...","[Built on a reclaimed swamp, two miles east of...","[Built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, ,, two, miles...","[built, on, a, reclaimed, swamp, two, miles, e...","[build, on, a, reclaim, swamp, two, mile, east...","[build, reclaim, swamp, two, mile, east, downt...",[[]],"[[(honolulu, GPE), (waikiki, GPE), (the ala wa...","[[Honolulu, Waikiki, the Ala Wai canal], [], [...","[[two miles, honolulu, waikiki, the ala wai ca...","[[lazy, day], [kalakaua, avenue], [waikiki], [...","[build, reclaim, swamp, mile, east, downtown, ...","[build, reclaim, swamp, east, downtown, island...","[swamp, island, city, sea, point, shop, narrow...","[island, city, point, garden, shop, narrow, se...","{'island': 3, 'city': 1, 'point': 1, 'garden':...","[(island, city), (city, point), (point, garden..."
1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...","[Folklore, films and widespread hearsay have g...","[Folklore, ,, films, and, widespread, hearsay,...","[folklore, ,, films, and, widespread, hearsay,...","[folklore, films, and, widespread, hearsay, ha...","[folklore, film, and, widespread, hearsay, hav...","[folklore, film, widespread, hearsay, give, fl...",[[]],"[[(cuba, GPE)], [(key west, GPE), (florida, GP...","[[Cuba], [Key West, Florida, USA, Keys], [the ...","[[hundred-mile, ninety miles, cuba, the 1980s]...","[[cuban], [ninety, mile], [today], [mile], [du...","[folklore, film, hearsay, florida, key, hundre...","[folklore, film, hearsay, chain, island, image...","[chain, island, beach, fishing, dive, coast, s...","[beach, bar, corner, boutique, dive, coast, se...","{'beach': 1, 'bar': 1, 'corner': 1, 'boutique'...","[(beach, bar), (bar, corner), (corner, boutiqu..."
2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,[America’s oldest and easily its most famous n...,"[America, ’, s, oldest, and, easily, its, most...","[america, ’, s, oldest, and, easily, its, most...","[america, s, oldest, and, easily, its, most, f...","[america, s, old, and, easily, its, most, famo...","[america, old, easily, famous, national, park,...",[[]],"[[(america, GPE), (yellowstone national park, ...","[[America, YELLOWSTONE NATIONAL PARK], [Yellow...","[[america, yellowstone national park, three mi...","[[dozen], [canyon], [yellowstone, lake], [thou...","[america, park, yellowstone, park, attract, vi...","[attract, visitor, reason, sheer, diversity, o...","[mountain, meadow, grizzly, bear, bison, buffa...","[course, coyote, dog, bison, wolf, elk, geyser...","{'course': 1, 'coyote': 1, 'dog': 1, 'bison': ...","[(course, coyote), (coyote, dog), (dog, bison)..."
3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,[Although the Big Island of Hawaii could hold ...,"[Although, the, Big, Island, of, Hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, the, big, island, of, hawaii, could...","[although, big, island, hawaii, could, hold, i...",[[]],"[[(the big island of hawaii, LOC)], [(oahu, GP...","[[the Big Island of Hawaii], [Oahu, Maui], [Ko...","[[the big island of hawaii, 185,000, half, cap...","[[eighty], [crater, rim, drive], [island], [oa...","[island, hawaii, hold, island, room, spare, po...","[hold, room, spare, population, town, people, ...","[hold, town, hold, fair, restauran

In [27]:
df.loc[25,'sent_without_NE']

['dwarf',
 'high-rise',
 'neighbours',
 'commission',
 'bishop',
 'reminder',
 'catholicism',
 'dominate',
 'city',
 'impress',
 'visit',
 'create',
 'replica',
 'church',
 'statue',
 'crown',
 'facade',
 'apostle',
 'statue',
 'atop',
 'cousin',
 'represent',
 'patron',
 'saint',
 'parish',
 'donate',
 'opulent',
 'altar',
 'marble',
 'onyx',
 'ivory',
 'surmount',
 'copper',
 'reproduction',
 'baldachin',
 'altar',
 'enter',
 'des',
 'contains',
 'relic',
 'include',
 'remains',
 'zoticus',
 'patron',
 'saint']

In [10]:
df['grams'] = df['no_NE_attractions_plain_unique_dict']+p.create_n_grams_to_list(df['no_NE_attractions_plain_unique'], 2)
df

C:\Users\jenni\.conda\envs\nlp_travel\lib\site-packages\pandas\core\computation\expressions.py:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return op(a, b)


TypeError: Cannot broadcast np.ndarray with operand of type <class 'list'>

In [30]:
from nltk.corpus import wordnet

In [37]:
synsets_list = wordnet.synsets('church')
synset_all_list=[]
for i in synsets_list:
    synset_all_list.extend(i.hypernym_paths())
print(synset_all_list)
synset_flat_list = []

for x in synset_all_list:
    synset_flat_list = synset_flat_list + x

synset_flat_list

[[Synset('entity.n.01'), Synset('abstraction.n.06'), Synset('group.n.01'), Synset('social_group.n.01'), Synset('organization.n.01'), Synset('institution.n.01'), Synset('religion.n.02'), Synset('church.n.01')], [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('artifact.n.01'), Synset('structure.n.01'), Synset('building.n.01'), Synset('place_of_worship.n.01'), Synset('church.n.02')], [Synset('entity.n.01'), Synset('abstraction.n.06'), Synset('psychological_feature.n.01'), Synset('event.n.01'), Synset('act.n.02'), Synset('activity.n.01'), Synset('ceremony.n.03'), Synset('religious_ceremony.n.01'), Synset('service.n.03'), Synset('church_service.n.01')], [Synset('entity.n.01'), Synset('abstraction.n.06'), Synset('group.n.01'), Synset('social_group.n.01'), Synset('body.n.02'), Synset('church.n.04')], [Synset('act.v.01'), Synset('perform.v.02'), Synset('church.v.01')]]


[Synset('entity.n.01'),
 Synset('abstraction.n.06'),
 Synset('group.n.01'),
 Synset('social_group.n.01'),
 Synset('organization.n.01'),
 Synset('institution.n.01'),
 Synset('religion.n.02'),
 Synset('church.n.01'),
 Synset('entity.n.01'),
 Synset('physical_entity.n.01'),
 Synset('object.n.01'),
 Synset('whole.n.02'),
 Synset('artifact.n.01'),
 Synset('structure.n.01'),
 Synset('building.n.01'),
 Synset('place_of_worship.n.01'),
 Synset('church.n.02'),
 Synset('entity.n.01'),
 Synset('abstraction.n.06'),
 Synset('psychological_feature.n.01'),
 Synset('event.n.01'),
 Synset('act.n.02'),
 Synset('activity.n.01'),
 Synset('ceremony.n.03'),
 Synset('religious_ceremony.n.01'),
 Synset('service.n.03'),
 Synset('church_service.n.01'),
 Synset('entity.n.01'),
 Synset('abstraction.n.06'),
 Synset('group.n.01'),
 Synset('social_group.n.01'),
 Synset('body.n.02'),
 Synset('church.n.04'),
 Synset('act.v.01'),
 Synset('perform.v.02'),
 Synset('church.v.01')]

In [ ]:
df_matrix = p.transaction_encoding(df['grams'].tolist())
df_matrix = p.dataframe_for_training(df_matrix, df["Place"])
df_matrix

In [ ]:
token_string_list = p.list_to_string(df["grams"])
df["tokens_string"] = [x.replace(" ", "_") for x in token_string_list]
df_matrix = p.use_tfidfVectorizer(df["tokens_string"])
df_matrix = p.dataframe_for_training(df_matrix, df["Place"])
df_matrix

### 1. Text Preprocessing <a class="anchor" id="Text_prep"></a>

In [2]:
prep_lemma = Preprocessing()
prep_lemma.preprocess_data('results_scrapping.csv', 'Lemmatization')

prep_stem = Preprocessing()
prep_stem.preprocess_data('results_scrapping.csv', 'Stemming')

C:\Users\jenni\.conda\envs\nlp_travel\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\jenni\.conda\envs\nlp_travel\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


### 2. Erstellen von Trainingsdaten <a class="anchor" id="Build_train_data"></a>

In [3]:
grams = ['1-grams','2-grams','mixed-grams']
matrix = ['TFIDF','True/False']
data_list = []

for g in grams:
    for m in matrix:
        data_list.append({'data':prep_lemma.create_train_data(g, m),
                          'wordreduction':'Lemmatization',
                          'grams':g,
                          'matrix':m})
        data_list.append({'data':prep_stem.create_train_data(g, m),
                          'wordreduction':'Stemming',
                          'grams':g,
                          'matrix':m})

ValueError: empty vocabulary; perhaps the documents only contain stop words

### 3. Clustering <a class="anchor" id="Clustering"></a>

In [ ]:
clustering = Clustering()
hyperparameter_dict = {"cluster_types":["KMeans","DBSCAN","AffinityPropagation","AgglomerativeClustering","Birch","BisectingKMeans","MiniBatchKMeans","MeanShift","OPTICS","SpectralClustering"],
                       "n_clusters":[2,3,4,5,6,7,8,9,10]}
df_results = clustering.run_clustering(data_list, hyperparameter_dict)

In [ ]:
df_results.to_csv('results_2a.csv')

### 4. Bonus: Automatisches mapping der NoNE zu Kategorien <a class="anchor" id="Bonus"></a>

In [ ]:
df_mapping_lemma = prep_lemma.df_mapping
df_mapping_stem = prep_stem.df_mapping

In [ ]:
df_mapping_lemma.to_csv('output/mapping_data_lemma.csv')
df_mapping_stem.to_csv('output/mapping_data_stem.csv')